# Get melt pool dynamics from videos.

Use the same code to track the melt pool for both ProtoShape videos.

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from imutils import contours
import imutils
import glob
import os
import pandas as pd
import decimal

In [2]:
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(11, 4)})

## Which video to open? Either Video 1 or Video 2.

In [3]:
VIDEO_NUMBER = 1

Depending on the video:
- set the correct video path
- set the correct output path for results
- set the build area region of interest
- set the start frame, as the printing starts a while after the video recording has started

In [4]:
if VIDEO_NUMBER == 1:
    video_file_path = "./in/video1/video1.mp4"
    results_output_path = "./out/protoshape/video1/processed_video/"
    # Video 1 build area region of interest
    [start_x, start_y, end_x, end_y] = [345, 180, 720, 590]
    # Start frame
    START_FRAME = 280
    
elif VIDEO_NUMBER == 2:
    video_file_path = "./in/video2/video2.avi"
    results_output_path = "./out/protoshape/video2/processed_video/"
    # Video 2 build area region of interest
    [start_x, start_y, end_x, end_y] = [274, 482, 586, 832]
    # Start frame
    START_FRAME = 6820

In [5]:
cap = cv2.VideoCapture(video_file_path)
if cap.isOpened() is False:
    print("Error opening video stream or file")

In [6]:
print("Total frame count of " + video_file_path + " is " + str(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

Total frame count of ./in/video1/video1.mp4 is 63948.0


## Get melt pool dynamics from each frame.

In [7]:
# Data frame for recording features.
df = pd.DataFrame(columns=['Frame_Index', 'Area', 'Mean', 'Radius', 'Rotated_Box'])

In [8]:
# TODO convert build_area_roi RGB to HSV for better light sensitivity?
# hsv1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)
# h, s, v1 = cv2.split(hsv1)

In [9]:
%%time
frame_index = 0

while cap.isOpened():
    if frame_index % 100 == 0:
        print("Processing frame " + str(frame_index))
    
    ret_val, image = cap.read()
    if image is None:
        break
   
    frame_index = frame_index + 1

    if frame_index < START_FRAME:
        continue

    build_area_roi = image[start_y:end_y, start_x:end_x]
    build_area_roi = cv2.cvtColor(build_area_roi, cv2.COLOR_BGR2GRAY)

    # Blur to remove noise (radius must be ODD)
    blur = cv2.GaussianBlur(build_area_roi, (5, 5), 0) # 21,21
    
    # Threshold the image to reveal light regions in the blurred image.
    thresh = cv2.threshold(blur, 210, 255, cv2.THRESH_BINARY)[1]

    # Perform a series of erosions and dilations to remove
    # any small blobs of noise from the thresholded image.
    thresh = cv2.erode(thresh, None, iterations=1)
    thresh = cv2.dilate(thresh, None, iterations=1)
    
    # Get contours.
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    if len(cnts) != 0:
        sorted_contours = sorted(cnts, key=cv2.contourArea, reverse=True)
        
        # Get top 2 contours (2nd largest if available) and combine their points to get encompassing rectangle.
        largest_contour = sorted_contours[0]
        top_contour_points = largest_contour
        
        # Get area of melt pool (largest + second largest if available).
        area = cv2.contourArea(largest_contour)
        
        if len(cnts) >= 2:
            second_largest_contour = sorted_contours[1]
            top_contour_points = np.concatenate((largest_contour,second_largest_contour))
            
            area = area + cv2.contourArea(second_largest_contour)

        # Get average intensity of melt pool.
        contour_mask = np.zeros(build_area_roi.shape, np.uint8)
        cv2.drawContours(contour_mask, top_contour_points, -1, 255, -1) # Draw filled contours.
        mean = cv2.mean(build_area_roi, mask=contour_mask)

        # Get radius of minimum enclosing circle of contour(s).
        ((cX, cY), radius) = cv2.minEnclosingCircle(top_contour_points)

        # Find rotated rectangle of encompassing contours. 
        rotated_rectangle = cv2.minAreaRect(top_contour_points)
        rotated_box = cv2.boxPoints(rotated_rectangle)
        rotated_box = np.int0(rotated_box)

        # Draw the rotated rectangle.
#         cv2.drawContours(image,[rotated_box],0,(0,0,255),2)
        
        df = df.append({'Frame_Index': f'{frame_index:07}', 'Area': area, 'Mean': mean[0], 'Radius': radius, 'Rotated_Box': rotated_box}, ignore_index=True)

    else:
        df = df.append({'Frame_Index': f'{frame_index:07}', 'Area': 0, 'Mean': 0, 'Radius': 0}, ignore_index=True)

    # Write frame
    #cv2.imwrite('./out/protoshape/video2/frames/frame' + '_' + f'{frame_index:07}' + '.png', roi)

# Release resources.
cap.release()

Processing frame 0
Processing frame 100
Processing frame 200
Processing frame 300
Processing frame 400
Processing frame 500
Processing frame 600
Processing frame 700
Processing frame 800
Processing frame 900
Processing frame 1000
Processing frame 1100
Processing frame 1200
Processing frame 1300
Processing frame 1400
Processing frame 1500
Processing frame 1600
Processing frame 1700
Processing frame 1800
Processing frame 1900
Processing frame 2000
Processing frame 2100
Processing frame 2200
Processing frame 2300
Processing frame 2400
Processing frame 2500
Processing frame 2600
Processing frame 2700
Processing frame 2800
Processing frame 2900
Processing frame 3000
Processing frame 3100
Processing frame 3200
Processing frame 3300
Processing frame 3400
Processing frame 3500
Processing frame 3600
Processing frame 3700
Processing frame 3800
Processing frame 3900
Processing frame 4000
Processing frame 4100
Processing frame 4200
Processing frame 4300
Processing frame 4400
Processing frame 4500


In [10]:
df.head()

,Frame_Index,Area,Mean,Radius,Rotated_Box
0,0000280,0,0,0,NaN
1,0000281,0,0,0,NaN
2,0000282,0,0,0,NaN
3,0000283,0,0,0,NaN
4,0000284,0,0,0,NaN


### Save pool dynamics to CSV file.

In [11]:
df.to_csv(results_output_path + "pool_data_from_video.csv",index=False)